[요구사항 1]
---
titanic_dataset.py 분석

https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/bg6gou1o?nw=nwuserajhajh503

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
from sklearn.preprocessing import LabelEncoder

In [2]:
class TitanicDataset(Dataset): # 데이터 셋을 상속받아 데이터를 load 하고 처리하기 쉽게 해줌
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X) # floattensor로 변환해 저장하고 feature와 target 받음.
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [3]:
class TitanicTestDataset(Dataset): # Test dataset에는 y 즉 target 값이 없으므로
  def __init__(self, X): # x(입력) 값만 처리
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

In [4]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    #all_df[["Pclass", "Fare"]] = 데이터셋에서 pclass, fare 열만 선택
    #groupby("Pclass") = pclass 열 기준으로 데이터를 그룹화
    #즉, 1,2,3 등급별로 그룹이 만들어짐.
    #mean()은 평균
    #reset_index()은 열로 변환하여 데이터프레임 형태로 만들어줌
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    
    Fare_mean.columns = ["Pclass", "Fare_mean"]#열이름을 pclass, fare_mean으로 변경
    #all_df 원래 데이터셋과 방금 만든 fare_mean 데이터셋을 결합
    #on="Pclass"열 기준으로 병합. left 즉 왼쪽 모든 데이터를 유지하면서 병합
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    #all_df["Fare"].isnull() Fare 값이 없는 데이터를 찾아
    #loc으로 맞는 행을 선택하고, 
    # #all_df["Fare"] = all_df["Fare_mean"] 없는 Fare 값을 Fare_mean 값으로 대체
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

In [5]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    #학습할 때 필요없는 특징
    #다만 나이의 평균 값을 구할 때, 존칭을 이용해 평균 나이값을 구하기 때문에
    #honorific은 의미 있는 특징이 됨
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [6]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # 존칭(Mr,Mrs)으로 그룹을 나눠 없는 데이터 셋에 평균 나이를 추가.
    #방식은 fare와 유사.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]

    # drop(["honorific_age_mean"], axis=1) = 해당 열 삭제.    
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [7]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    #가족의 수 = 부모와 자녀 + 형제자매와 배우자 수
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    #가족수가 0이면 alone 새 열을 만들어 1 할당
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    #alone 값이 0 즉 결측값이면 0으로 채워줌
    #all_df["alone"].fillna(0, inplace=True)
    all_df["alone"] = all_df["alone"].fillna(0)

    # 학습에 불필요한 컬럼 제거 
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [8]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # ~(...)은 부정 연산자로 안에 조건이 들어감.
    # 조건을 만족하지 않는 경우 other로 대체됨
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"

    #embarked 값이 없는경우 missing이라는 값을 할당. 
    #all_df["Embarked"].fillna("missing", inplace=True)
    all_df["Embarked"] = all_df["Embarked"].fillna("missing")
    return all_df

In [9]:
#카테고리별 변수들을 수치화 값으로 변환하는 함수

def get_preprocessed_dataset_6(all_df):
    # 범주형 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # all_df.columns 모든 열을 반환해서
    #object 즉 문자열인 열만 모아서 category_features에 저장
    category_features = all_df.columns[all_df.dtypes == "object"]
    #범주형 데이터를 수치형으로 반환해주는 라이브러리
    #각 범주형 데이터들을 고유한 정수로 변환해주는 for문
    for category_feature in category_features: # 범주형 데이터들을 하나씩 뽑아서
        
        le = LabelEncoder()
        #범주형 데이터면
        if all_df[category_feature].dtypes == "object":
          #고유한 정수로 맵핑
          le = le.fit(all_df[category_feature])
          #범주형 데이터를 수치형으로 변환.
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

In [10]:
#데이터의 전처리와 분할을 담당하는 파트.
def get_preprocessed_dataset():
    #CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    # __file__변수는 스크립트 파일 경로이지만 쥬피터 노트북으로 변경했기 때문에
    #자동으로 정의되지 않음. 따라서 현재 작업 디렉토리를 직접 설정해줘야함.
    CURRENT_FILE_PATH = os.getcwd()  # 현재 디렉토리 경로 사용
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv") # 파일을 연결
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path) # 데이터 load
    test_df = pd.read_csv(test_data_path)

    #한번에 전처리 하기 위해서 단순히 뒤에 이어 붙이는 concat 사용
    #train에는 label인 survived가 있지만 test에는 없는걸 고려해서 처리해야함. 
    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df) #fare 결측값 처리리

    all_df = get_preprocessed_dataset_2(all_df) #이름 분리

    all_df = get_preprocessed_dataset_3(all_df) # age 결측값 추가가

    all_df = get_preprocessed_dataset_4(all_df) # 가족, 필요없는 특징 제거

    all_df = get_preprocessed_dataset_5(all_df) # embarked 결측값 추가 및 honorific 값 개수 줄이기

    all_df = get_preprocessed_dataset_6(all_df) # 범주형 특징값들을 고유한 수치로 맵핑

    #concat으로 붙여놨던 train 데이터와 test 데이터를 분리하는 작업
    # train 데이터를 validation 데이터도 분리
    #survived 값이 결측값 = test 데이터 이므로 부정 연산자를 사용해
    # survived 값이 있는 데이터를 train x에 추가.
    #drop을 사용해 target 인 survived를 제거
    #reset_index(drop=True) 기존 인덱스에 새로운 열을 추가하지 않고 완전 제거 후 인덱스 재설정
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    # ~ 부정 연산자만 빼서 survived가 없는 데이터를 test x에 추가.
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 입력과 출력(target)으로 데이터 셋 구성
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    #데이터 셋을 8:2로 train 데이터와 validation 데이터를 분리
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)
    # 데이터 셋을 train, validation, test로 분리후 반환
    return train_dataset, validation_dataset, test_dataset

In [11]:
#모델 구조 정의하는 파트

class MyModel(nn.Module):
  #n_input = 입력 특징의 개수
  #n_output = survived 여부 즉 0 or 1
  def __init__(self, n_input, n_output): # 모델 구조 정의
    super().__init__()

    #순차적으로 레이어를 정의
    #FCL이고 활성화 함수는 Relu 사용
    # x의 특징 수 11를 입력으로 받아
    # 30개로 선형 변환 후 히든레이어 하나 더 지나 0 or 1이 출력됨
    self.model = nn.Sequential(
      nn.Linear(n_input, 30),
      nn.ReLU(),
      nn.Linear(30, 30),
      nn.ReLU(),
      nn.Linear(30, n_output),
    )

  def forward(self, x):
    x = self.model(x)
    return x

In [12]:
def test(test_data_loader):
  print("[TEST]")
  #데이터를 배치 단위로 반복
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  #print("{0}".format(batch['input']))
  # 특징 수 11개와 0 or 1을 출력하는 모델 생성
  my_model = MyModel(n_input=11, n_output=2)

  #입력데이터를 넣어 예측을 수행.
  output_batch = my_model(batch['input'])
  #dim 1 즉, 0 or 1 중 더 예측값이 높은거 하나 선택
  prediction_batch = torch.argmax(output_batch, dim=1)

  #예측한 값 출력
  #for idx, prediction in enumerate(prediction_batch, start=892):
  #    print(idx, prediction.item())

In [13]:
if __name__ == "__main__": # 실행
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)
  # 잘 만들어진 데이터를 load
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000, 32.0000,  2.0000,  0.0000, 73.5000,  2.0000, 21.1792,
         2.0000,  2.0000,  0.0000]): 0
1 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
2 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 1.0000,  1.0000, 36.0000,  0.0000,  0.0000, 40.1250,  0.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.2250,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  0.0000, 31.0000,  0.0000,  0.0000,  7.8542,  2.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 0
6 - tensor([ 3.0000,  1.0000, 55.5000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
7 - tensor([ 2.00

[요구사항 2]
----
titanic 딥러닝 모델 훈련 코드 및 activation function 변경해보기

In [14]:
from torch import optim
from datetime import datetime
import wandb
import sys

In [15]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        
        self.model = nn.Sequential(
            #wandb.config.n_hidden_unit_list[0] 에서 설정된 첫 번째 히든레이어 노드 수
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [16]:
def get_model_and_optimizer():
    my_model = MyModel(n_input=11, n_output=2)

    #스토캐스틱 그라디언트 디센트로 모델 학습, 러닝 레이트는 wandb.config.learning_rate
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)
    return my_model, optimizer

In [17]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs # 반복 횟수
    loss_fn = nn.CrossEntropyLoss()  # 이진 분류 문제이므로 CrossEntropyLoss 사용
    next_print_epoch = 100 # 100번마다 loss 출력

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            #target을 예측하는 학습 수행
            input, target = train_batch['input'], train_batch['target']
            output_train = model(input)
            #loss 계산
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1

            #그라디언트 초기화, 그라디언트가 초기화를 자동으로 안해줌
            optimizer.zero_grad()
            # 그라디언트 계산
            loss.backward()
            #SGD
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0

        #computation graph x
        #validation 계산, train과 동일한 알고리즘이지만 업데이트는 하지 않음.
        with torch.no_grad():
            for validation_batch in validation_data_loader:
                input, target = validation_batch['input'], validation_batch['target']
                output_validation = model(input)
                loss = loss_fn(output_validation, target)
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })
        #100회마다 진행
        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100

In [18]:
#모델을 test 하고 submission 생성

def test_and_create_submission(model, test_data_loader):
    model.eval()  # 모델을 평가 모드로 전환, 가중치를 업데이트 하지 않음
    all_predictions = []
    passenger_ids = list(range(892, 892 + len(test_data_loader.dataset)))  # Titanic 테스트 데이터의 승객 ID는 892부터 시작

    #업데이트를 하지 않기 때문에 no_grad
    with torch.no_grad():
        for test_batch in test_data_loader:
            input = test_batch['input']  # TitanicTestDataset을 사용하여 'input' 데이터를 가져옴
            output = model(input)
            predictions = torch.argmax(output, dim=1).cpu().numpy()  # 0 or 1 중 가장 높은 확률의 클래스를 예측
            all_predictions.extend(predictions)

    # submission.csv 생성
    submission_df = pd.DataFrame({
        #형식에 맞게
        'PassengerId': passenger_ids,
        'Survived': all_predictions
    })
    
    submission_df.to_csv('submission_test.csv', index=False)
    print("submission_test.csv 파일이 생성되었습니다!")

In [19]:
def main(args):
    #현재 시간 기록
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    #모델에 설정할 값들을 담은 딕셔너리
    #에포크, 배치사이즈, 러닝 레이트, 히든 유닛 개수 등 포함되어 있음
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    #wandb 모델의 초기값 생성
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="Titanic Dataset experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    #모델과 옵티마이저 생성
    model, optimizer = get_model_and_optimizer()

    print("#" * 50, 1)

    #모델 train 시작.
    training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )
    #train 끝
    #test 실행
    test_and_create_submission(model, test_data_loader)

    wandb.finish()

In [20]:
if __name__ == "__main__":
    # Jupyter Notebook 환경을 확인하여 argparse를 우회.
    if "ipykernel_launcher" in sys.argv[0]:
        # 기본값을 설정
        class Args:
            wandb = True
            batch_size = 512
            epochs = 2000 # Early stopping을 위해 epochs를 2000으로 설정

        args = Args() # args 추가
        main(args) # 실행

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ajhajh503 (ajhajh503-korea-university-of-technology-and-education). Use `wandb login --relogin` to force relogin


{'epochs': 2000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5617, Validation loss 0.5655
Epoch 200, Training loss 0.5521, Validation loss 0.5916
Epoch 300, Training loss 0.5294, Validation loss 0.5316
Epoch 400, Training loss 0.5060, Validation loss 0.5474
Epoch 500, Training loss 0.4754, Validation loss 0.5379
Epoch 600, Training loss 0.4517, Validation loss 0.4871
Epoch 700, Training loss 0.4382, Validation loss 0.5334
Epoch 800, Training loss 0.4189, Validation loss 0.4767
Epoch 900, Training loss 0.4232, Validation loss 0.4773
Epoch 1000, Training loss 0.4232, Validation loss 0.4789
Epoch 1100, Training loss 0.4054, Validation loss 0.5427
Epoch 1200, Training loss 0.4059, Validation loss 0.5976
Epoch 1300, Training loss 0.4005, Validation loss 0.4707
Epoch 1400, Training loss 0.3926, Validation loss 0.4882
Epoch 1500, Training loss 0.3907, Validation loss 0.5508
Epoch 160

Epoch,▁▁▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇████
Training loss,█████▇▇▇▆▆▅▄▄▃▃▃▃▃▃▃▂▃▃▃▂▃▂▂▃▂▂▂▂▁▂▂▁▁▂▁
Validation loss,▅▅▆▅▄▅▅▂▃▄▄▂▃▃▂▁▂▁▂▃▇▃▂▁▁▂▁▂▂▁▅▃▂▄▂▃▃▁▄█
Epoch,2000
Training loss,0.38812
Validation loss,0.608


Relu 활성화 함수
---
https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/77qgbsbu?nw=nwuserajhajh503

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Relu_training_loss.svg)
![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Relu_V.svg)

In [21]:
#ELU로 변경, 그외 전부 동일

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        
        self.model = nn.Sequential(
            #wandb.config.n_hidden_unit_list[0] 에서 설정된 첫 번째 히든레이어 노드 수
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ELU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ELU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

if __name__ == "__main__":
    # Jupyter Notebook 환경을 확인하여 argparse를 우회.
    if "ipykernel_launcher" in sys.argv[0]:
        # 기본값을 설정
        class Args:
            wandb = True
            batch_size = 512
            epochs = 2000 # Early stopping을 위해 epochs를 2000으로 설정

        args = Args() # args 추가
        main(args) # 실행

{'epochs': 2000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5589, Validation loss 0.5985
Epoch 200, Training loss 0.5267, Validation loss 0.5495
Epoch 300, Training loss 0.4993, Validation loss 0.6037
Epoch 400, Training loss 0.4664, Validation loss 0.5006
Epoch 500, Training loss 0.4634, Validation loss 0.5793
Epoch 600, Training loss 0.4379, Validation loss 0.5993
Epoch 700, Training loss 0.4257, Validation loss 0.5441
Epoch 800, Training loss 0.4109, Validation loss 0.5218
Epoch 900, Training loss 0.4007, Validation loss 0.4736
Epoch 1000, Training loss 0.3959, Validation loss 0.5473
Epoch 1100, Training loss 0.3990, Validation loss 0.4739
Epoch 1200, Training loss 0.4033, Validation loss 0.5078
Epoch 1300, Training loss 0.3954, Validation loss 0.4806
Epoch 1400, Training loss 0.3860, Validation loss 0.5268
Epoch 1500, Training loss 0.3813, Validation loss 0.5453
Epoch 160

Epoch,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇████
Training loss,█▇▇▄▄▅▄▄▃▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Validation loss,▄▄▃▄▃▃▄▃▁▃▂▂▂▅▂▂▇▂▁▁▄▁▂▅▁▁▂▂▂▅▃▄▃▁▁▁█▂▃▂
Epoch,2000
Training loss,0.36293
Validation loss,0.56218


ELU 활성화 함수
---
https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/qazpje1y?nw=nwuserajhajh503

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/ELU_T.svg)
![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/ELU_V.svg)

In [22]:
#LeakyReLU로 변경, 그외 전부 동일

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        
        self.model = nn.Sequential(
            #wandb.config.n_hidden_unit_list[0] 에서 설정된 첫 번째 히든레이어 노드 수
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.LeakyReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.LeakyReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

if __name__ == "__main__":
    # Jupyter Notebook 환경을 확인하여 argparse를 우회.
    if "ipykernel_launcher" in sys.argv[0]:
        # 기본값을 설정
        class Args:
            wandb = True
            batch_size = 512
            epochs = 2000 # Early stopping을 위해 epochs를 2000으로 설정

        args = Args() # args 추가
        main(args) # 실행

{'epochs': 2000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5718, Validation loss 0.5917
Epoch 200, Training loss 0.5533, Validation loss 0.6267
Epoch 300, Training loss 0.5342, Validation loss 0.5681
Epoch 400, Training loss 0.4921, Validation loss 0.5394
Epoch 500, Training loss 0.4626, Validation loss 0.5517
Epoch 600, Training loss 0.4328, Validation loss 0.6449
Epoch 700, Training loss 0.4255, Validation loss 0.4834
Epoch 800, Training loss 0.4306, Validation loss 0.5361
Epoch 900, Training loss 0.4231, Validation loss 0.4775
Epoch 1000, Training loss 0.4020, Validation loss 0.5486
Epoch 1100, Training loss 0.3912, Validation loss 0.6462
Epoch 1200, Training loss 0.4127, Validation loss 0.6023
Epoch 1300, Training loss 0.4005, Validation loss 0.5501
Epoch 1400, Training loss 0.4035, Validation loss 0.8314
Epoch 1500, Training loss 0.3953, Validation loss 0.4848
Epoch 160

Epoch,▁▁▁▁▂▂▂▂▂▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇████
Training loss,███▇▇▇▆▆▆▆▅▅▅▄▄▃▃▄▃▃▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▁▁▁▁▁
Validation loss,▄▄▅▄▄▄▃█▄▂▂▂▃▃▁▃▃▃▁▂▂▂▆▂▂▂▅▃▂▂▁▁▂▁▁▃▁▄▁▄
Epoch,2000
Training loss,0.37016
Validation loss,0.67026


LeakyReLU 활성화 함수
---
https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/vzofv6wt?nw=nwuserajhajh503

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Leaky_T.svg)
![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Leaky_V.svg)

In [23]:
#PReLU로 변경, 그외 전부 동일

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        
        self.model = nn.Sequential(
            #wandb.config.n_hidden_unit_list[0] 에서 설정된 첫 번째 히든레이어 노드 수
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.PReLU(num_parameters=1, init=0.25),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.PReLU(num_parameters=1, init=0.25),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

if __name__ == "__main__":
    # Jupyter Notebook 환경을 확인하여 argparse를 우회.
    if "ipykernel_launcher" in sys.argv[0]:
        # 기본값을 설정
        class Args:
            wandb = True
            batch_size = 512
            epochs = 2000 # Early stopping을 위해 epochs를 2000으로 설정

        args = Args() # args 추가
        main(args) # 실행

{'epochs': 2000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5668, Validation loss 0.5775
Epoch 200, Training loss 0.5525, Validation loss 0.6186
Epoch 300, Training loss 0.5248, Validation loss 0.5759
Epoch 400, Training loss 0.5058, Validation loss 0.5006
Epoch 500, Training loss 0.4826, Validation loss 0.4879
Epoch 600, Training loss 0.4659, Validation loss 0.5259
Epoch 700, Training loss 0.4404, Validation loss 0.4956
Epoch 800, Training loss 0.4320, Validation loss 0.5646
Epoch 900, Training loss 0.4256, Validation loss 0.4869
Epoch 1000, Training loss 0.4396, Validation loss 0.4716
Epoch 1100, Training loss 0.4278, Validation loss 0.5464
Epoch 1200, Training loss 0.4061, Validation loss 0.4723
Epoch 1300, Training loss 0.4011, Validation loss 0.4616
Epoch 1400, Training loss 0.3855, Validation loss 0.4764
Epoch 1500, Training loss 0.3840, Validation loss 0.5716
Epoch 160

Epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
Training loss,███▇▇▇▇▇▆▇▅▄▄▅▄▄▃▄▃▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁
Validation loss,▅▅▄▅▅▅▅▄▅▄▃▅▅▂█▃▄▁▂▂▁▁▁▁▄▂▄▂▁█▆▁▂▂▃▁▃▃▇▅
Epoch,2000
Training loss,0.35018
Validation loss,0.52537


PReLU 활성화 함수
---
https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/3mhrerpw?nw=nwuserajhajh503

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/PRelu_T.svg)
![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/PRelu_V.svg)

[요구 사항3]
---
테스트 및 submission.csv 생성

오버피팅으로 인해 Activation Function 차이를 보기 힘들었음.

따라서 제일 가성비 좋은 기본 Relu를 사용해서 모델을 구성하고, 수업시간에 배운 Early Stopping을 사용해

Validation이 지속해서 증가하면 훈련을 멈추게 구성

In [24]:
def training_loop(model, optimizer, train_data_loader, validation_data_loader, args):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()
    best_validation_loss = float('inf')
    patience_counter = 0  # Early stopping을 위한 patience 카운터
    next_print_epoch = 100 # 100번마다 출력

    for epoch in range(1, n_epochs + 1):
        model.train()  # 학습 모드 전환
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            input, target = train_batch['input'], train_batch['target']
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % args.validation_intervals == 0:  # 10번마다 검증 실행
            model.eval()  # 평가 모드 전환
            loss_validation = 0.0
            num_validations = 0
            with torch.no_grad():
                for validation_batch in validation_data_loader:
                    input, target = validation_batch['input'], validation_batch['target']
                    output_validation = model(input)
                    loss = loss_fn(output_validation, target)
                    loss_validation += loss.item()
                    num_validations += 1

            avg_validation_loss = loss_validation / num_validations
            wandb.log({
                "Epoch": epoch,
                "Training loss": loss_train / num_trains,
                "Validation loss": avg_validation_loss
            })

            # Early stopping 조건 확인
            if avg_validation_loss + 0.00001< best_validation_loss:
                best_validation_loss = avg_validation_loss
                patience_counter = 0  # 성능이 개선되면 patience 초기화
            else:
                patience_counter += 1  # 개선되지 않으면 카운터 증가

            if patience_counter >= args.early_stop_patience:  # patience 초과 시 학습 종료
                print(f"Early stopping at epoch {epoch}. Best validation loss: {best_validation_loss:.4f}")
                break
            if epoch >= next_print_epoch:
                print(
                    f"Epoch {epoch}, "
                    f"Training loss {loss_train / num_trains:.4f}, "
                    f"Validation loss {loss_validation / num_validations:.4f}"
                )
                next_print_epoch += 100

class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        
        self.model = nn.Sequential(
            #wandb.config.n_hidden_unit_list[0] 에서 설정된 첫 번째 히든레이어 노드 수
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

def main(args):
    #현재 시간 기록
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    #모델에 설정할 값들을 담은 딕셔너리
    #에포크, 배치사이즈, 러닝 레이트, 히든 유닛 개수 등 포함되어 있음
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [20, 20],
    }

    #wandb 모델의 초기값 생성
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="Titanic Dataset experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    #모델과 옵티마이저 생성
    model, optimizer = get_model_and_optimizer()

    print("#" * 50, 1)

    #모델 train 시작.
    training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
        args=args # args 추가
    )
    #train 끝
    #test 실행
    test_and_create_submission(model, test_data_loader)

    wandb.finish()

if __name__ == "__main__":
    if "ipykernel_launcher" in sys.argv[0]:
        # 기본값을 설정
        class Args:
            wandb = True
            batch_size = 512
            epochs = 2000 # Early stopping을 위해 epochs를 2000으로 설정
            validation_intervals = 10
            early_stop_patience = 20

        args = Args() # args 추가
        main(args) # 실행

{'epochs': 2000, 'batch_size': 512, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
################################################## 1
Epoch 100, Training loss 0.5717, Validation loss 0.5569
Epoch 200, Training loss 0.5581, Validation loss 0.5758
Epoch 300, Training loss 0.5438, Validation loss 0.5753
Epoch 400, Training loss 0.5261, Validation loss 0.6289
Epoch 500, Training loss 0.4998, Validation loss 0.5295
Epoch 600, Training loss 0.4732, Validation loss 0.4898
Epoch 700, Training loss 0.4526, Validation loss 0.5931
Epoch 800, Training loss 0.4337, Validation loss 0.4789
Epoch 900, Training loss 0.4142, Validation loss 0.5368
Epoch 1000, Training loss 0.4186, Validation loss 0.5575
Early stopping at epoch 1070. Best validation loss: 0.4487
submission_test.csv 파일이 생성되었습니다!


Epoch,▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇███
Training loss,████▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▄▃▄▃▃▃▃▂▃▂▂▃▂▂▂▂▂▁▁▁
Validation loss,▆▇▆▇▆▇▆▅▄▅▆█▅▄▅▄▅▅▄▃▃▇▂▃▃▂▂▂▃▂▅▄▃▃▁▁▅▃▃▆
Epoch,1070
Training loss,0.41197
Validation loss,0.57084


Early-Stop
---
https://wandb.ai/ajhajh503-korea-university-of-technology-and-education/my_model_training/runs/dybp8t2f?nw=nwuserajhajh503

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Early_T.svg)
![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/Early_V.svg)

[요구 사항4]
---
submission.csv 제출 및 등수확인

오버피팅을 줄이기 위해 다양한 정규화 추가
변경된 사항으로는 히든레이어 추가, 옵티마이저 변경, 배치 정규화, 드롭 아웃, 하이퍼파라미터 조정 등 변경

In [25]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),  # n_input을 실제 입력 크기로 수정
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[0]), # batch_normalziation 추가
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Dropout(0.3),

            #히든 레이어 추가
            nn.Linear(wandb.config.n_hidden_unit_list[1], wandb.config.n_hidden_unit_list[2]),
            nn.BatchNorm1d(wandb.config.n_hidden_unit_list[2]),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(wandb.config.n_hidden_unit_list[2], n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

def get_model_and_optimizer():
    n_input = 11
    n_output = 2
    model = MyModel(n_input=n_input, n_output=n_output)

    #SGD에서 ADAM으로 변경
    optimizer = optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
    return model, optimizer

def training_loop(model, optimizer, train_data_loader, validation_data_loader, args):
    n_epochs = wandb.config.epochs
    loss_fn = nn.CrossEntropyLoss()
    best_validation_loss = float('inf')
    patience_counter = 0  # Early stopping을 위한 patience 카운터
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        model.train()  # 학습 모드 전환
        loss_train = 0.0
        num_trains = 0
        for train_batch in train_data_loader:
            input, target = train_batch['input'], train_batch['target']
            output_train = model(input)
            loss = loss_fn(output_train, target)
            loss_train += loss.item()
            num_trains += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % args.validation_intervals == 0:  # 10번마다 검증 실행
            model.eval()  # 평가 모드 전환
            loss_validation = 0.0
            num_validations = 0
            with torch.no_grad():
                for validation_batch in validation_data_loader:
                    input, target = validation_batch['input'], validation_batch['target']
                    output_validation = model(input)
                    loss = loss_fn(output_validation, target)
                    loss_validation += loss.item()
                    num_validations += 1

            avg_validation_loss = loss_validation / num_validations
            wandb.log({
                "Epoch": epoch,
                "Training loss": loss_train / num_trains,
                "Validation loss": avg_validation_loss
            })

            # Early stopping 조건 확인
            if avg_validation_loss + 0.00001< best_validation_loss:
                best_validation_loss = avg_validation_loss
                patience_counter = 0  # 성능이 개선되면 patience 초기화
            else:
                patience_counter += 1  # 개선되지 않으면 카운터 증가

            if patience_counter >= args.early_stop_patience:  # patience 초과 시 학습 종료
                print(f"Early stopping at epoch {epoch}. Best validation loss: {best_validation_loss:.4f}")
                break
            if epoch >= next_print_epoch:
                print(
                    f"Epoch {epoch}, "
                    f"Training loss {loss_train / num_trains:.4f}, "
                    f"Validation loss {loss_validation / num_validations:.4f}"
                )
                next_print_epoch += 100

def test_and_create_submission(model, test_data_loader):
    model.eval()  # 모델을 평가 모드로 전환
    all_predictions = []
    passenger_ids = list(range(892, 892 + len(test_data_loader.dataset)))  # Titanic 테스트 데이터의 승객 ID는 892부터 시작

    with torch.no_grad():
        for test_batch in test_data_loader:
            input = test_batch['input']  # TitanicTestDataset을 사용하여 'input' 데이터를 가져옴
            output = model(input)
            predictions = torch.argmax(output, dim=1).cpu().numpy()  # 가장 높은 확률의 클래스를 예측
            all_predictions.extend(predictions)

    # submission.csv 생성
    submission_df = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Survived': all_predictions
    })
    submission_df.to_csv('submission_test.csv', index=False)
    print("submission_test.csv 파일이 생성되었습니다!")

def main(args):
    #현재 시간 기록
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    #모델에 설정할 값들을 담은 딕셔너리
    #에포크, 배치사이즈, 러닝 레이트, 히든 유닛 개수 등 포함되어 있음
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 5e-4,
        'n_hidden_unit_list': [64, 32, 16],
    }

    #wandb 모델의 초기값 생성
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="Titanic Dataset experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    #모델과 옵티마이저 생성
    model, optimizer = get_model_and_optimizer()

    print("#" * 50, 1)

    #모델 train 시작.
    training_loop(
        model=model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader,
        args=args # args 추가
    )
    #train 끝
    #test 실행
    test_and_create_submission(model, test_data_loader)

    wandb.finish()

if __name__ == "__main__":
    if "ipykernel_launcher" in sys.argv[0]:
        class Args:
            wandb = True
            batch_size = 512
            epochs = 1500 # Early stopping을 위해 epochs를 1000으로 설정
            validation_intervals = 10
            early_stop_patience = 50

        args = Args() # args 추가
        main(args) # 실행

{'epochs': 1500, 'batch_size': 512, 'learning_rate': 0.0005, 'n_hidden_unit_list': [64, 32, 16]}
################################################## 1
Epoch 100, Training loss 0.4345, Validation loss 0.5794
Epoch 200, Training loss 0.3957, Validation loss 0.4565
Epoch 300, Training loss 0.3987, Validation loss 0.4923
Epoch 400, Training loss 0.3957, Validation loss 0.4495
Epoch 500, Training loss 0.3897, Validation loss 0.4936
Epoch 600, Training loss 0.3942, Validation loss 0.4515
Epoch 700, Training loss 0.3785, Validation loss 0.4574
Epoch 800, Training loss 0.3671, Validation loss 0.5478
Early stopping at epoch 900. Best validation loss: 0.4495
submission_test.csv 파일이 생성되었습니다!


Epoch,▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇██
Training loss,█▅▅▄▄▄▃▂▃▃▂▃▃▃▃▂▃▃▂▂▃▂▂▂▂▂▂▂▁▂▁▃▂▁▃▂▁▁▂▁
Validation loss,█▆▂▃▄▂▁▃▄▁▁▁▃▁▂▁█▇▁▁▅▃▃▅▁▁▁▃▁▂▁▂▅▃▂▄▂▅▂▂
Epoch,900
Training loss,0.3679
Validation loss,0.46997


submission
---

![Relu Activation Function](https://raw.githubusercontent.com/ajh1004ajh00/link_dl/main/_04_homeworks_solution/image.png)

[숙제 후기]
---
titanic dataset을 이용해 모델을 직접 설계해보는 실습을 해 보았습니다.

제일 기억에 남는것은 Loss는 감소하는 반면 validation Loss는 개선되지 않는 오버피팅 현상을 겪었습니다.

이를 해결하기 위해 Batch Normalization, Dropout, 하이퍼파라미터 조정 등 다양하게 해 보았지만 성능차이가 크게 나지 않았습니다.

처음보단 오버피팅을 완화하긴 했지만 그 차이가 크지 않아 아쉬웠습니다.

이번 과제를 통해 딥러닝 모델의 학습 과정에서 발생할 수 있는 문제들을 이해하고,
이를 해결하기 위한 다양한 기법들을 실제로 적용해 볼 수 있었습니다.
또한, 하이퍼파라미터 튜닝이 진짜 어렵다는 것을 알게된 시간이였습니다.